In [1]:
import arcpy, math
import os
import os.path
from arcpy import env
import random
from arcpy.sa import *
import pandas as pd
import numpy as np
# Check out any necessary licenses
arcpy.CheckOutExtension("3D")
arcpy.CheckOutExtension("spatial")
import matplotlib.pyplot as plt

ImportError: No module named arcpy

In [2]:
folder_nadrzedny_lista =[]
#folder_nadrzedny_lista = [r'c:\doktorat\czemp6\kondracki_mezo',
#                            r'c:\doktorat\czemp6\kondracki_mezo_makro',
#                             r'c:\doktorat\czemp6\powiaty',
#                          r'c:\doktorat\czemp6\kwadraty_losowe',
#                         'c:\doktorat\czemp6\kwadraty_10km']

                
for folder_nadrzedny in folder_nadrzedny_lista:                     
    arcpy.env.workspace = folder_nadrzedny
    model_folders = arcpy.ListWorkspaces()
    tabela_wspolczynnikow=pd.DataFrame

    lista_csv =[]

    for model_folder in model_folders:
            data_frame = pd.DataFrame()
            arcpy.env.workspace = model_folder
            os.chdir(model_folder)
            lista_rastrow = arcpy.ListRasters()
            arcpy.env.overwriteOutput = True
            lista_kolumn =[]
            for j in lista_rastrow:
                try:
                    array1 = arcpy.RasterToNumPyArray(j)
                    plt.imshow(array1)
                    plt.show()
                    plt.clf()
                    array_plaska = array1.ravel()
                    series = pd.Series(array_plaska)

                    #series.rename(nazwa_series)
                    kolumna = str(j[:-4])
                    data_frame[kolumna] = series.values
                    lista_kolumn.append(kolumna)
                except:
                    print("dasd")

            df_skrocony2 = data_frame.sample(frac=0.25, replace=True)
            df_skrocony = df_skrocony2.replace({255: np.NaN, -32768.000000: np.NaN, -9999: np.NaN})
            df3 = df_skrocony.dropna()
            csv = os.path.join(folder_nadrzedny, str(model_folder) + ".csv")
            print(csv)
            df3.to_csv(csv)

            lista_csv.append(df3)